In [6]:
import os


train_split = '/data2/itri/DCV/single_sensor_data/bag2/_front/radar_cart_train_test_split/train'
val_split = '/data2/itri/DCV/single_sensor_data/bag2/_front/radar_cart_train_test_split/val'
test_split = '/data2/itri/DCV/single_sensor_data/bag2/_front/radar_cart_train_test_split/test'

json_anno_path = '/data2/itri/DCV/single_sensor_data/bag2/_front_angle/labelme_json'


train = sorted(os.listdir(train_split))
val = sorted(os.listdir(val_split))
test = sorted(os.listdir(test_split))

json_anno = sorted(os.listdir(json_anno_path))
out_json_path = '/data2/itri/DCV/single_sensor_data/bag2/_front_angle/annotations'

In [2]:

img_idx = 1
anno_idx = 1

annotations = []
images = []
categories = [
 {'id': 1, 'name': 'car'},
 {'id': 2, 'name': 'bus'},
 {'id': 3, 'name': 'truck'},
 {'id': 4, 'name': 'pedestrian'},
 {'id': 5, 'name': 'bimo'}]


In [54]:
import json
for i, f in enumerate(list(map(lambda x: int(x[:-5]), train))):
	with open(os.path.join(json_anno_path, json_anno[i])) as fl:
		data = json.load(fl)
	

	images.append(
		{
      "file_name": "train/{}.png".format(f),
      "id": img_idx
    })

	
	# print(data['shapes'])
	single_img_anno = [x['points'] for x in data['shapes']]
	# print(single_img_anno)

	single_img_cls = [x['label'] for x in data['shapes']]
	# print(single_img_cls)


	for anno, cls in zip(single_img_anno, single_img_cls):
		x_min = min(anno[0][0], anno[1][0])
		y_min = min(anno[0][1], anno[1][1])
		x_max = max(anno[0][0], anno[1][0])
		y_max = max(anno[0][1], anno[1][1])

		w = x_max - x_min
		h = y_max - y_min
		annotations.append(
			{
				"id": anno_idx,
				"category_id": cls + 1,
				"image_id": img_idx,
				"bbox": [
					x_min,
					y_min,
					w,
					h,
					0
				],
				"conf": 1,
				"iscrowd": 0,
				"area": w*h
			})
		anno_idx += 1

	img_idx += 1


In [55]:
out = dict(
	images=images,
	annotations=annotations,
	categories=categories
)

In [56]:
with open('tmp.json', "w") as f:
	json.dump(out, f, indent = 2)

### gen train, val, test split anno from labelme.json (lidar detection result)
#### run this block and first block is enough

In [7]:
import json

split = [train, val, test]
split_str = ["train", "val", "test"]


for si, s in enumerate(split):
	img_idx = 1
	anno_idx = 1

	annotations = []
	images = []
	# categories = [
	# 	{'id': 1, 'name': 'car'},
	# 	{'id': 2, 'name': 'bus'},
	# 	{'id': 3, 'name': 'truck'},
	# 	{'id': 4, 'name': 'pedestrian'},
	# 	{'id': 5, 'name': 'bimo'}]

	categories = [
		{'id': 1, 'name': 'car'},
		{'id': 2, 'name': 'bus_truck'},
		{'id': 3, 'name': 'ped_bimo'}]
	
	cat_cvt = {
		0: 0,
		1: 1,
		2: 1,
		3: 2,
		4: 2
	}

	for i, f in enumerate(list(map(lambda x: int(x[:-4]), s))):
		try:
			with open(os.path.join(json_anno_path, '{}.json'.format(f))) as fl:
				data = json.load(fl)
		except:
			continue

		
		images.append(
			{
				"file_name": "{}/{}.png".format(split_str[si], f),
				"id": img_idx
			})

		
		# print(data['shapes'])
		single_img_anno = [x['points'] for x in data['shapes']]
		# print(single_img_anno)

		single_img_cls = [x['label'] for x in data['shapes']]
		# print(single_img_cls)

		# if f == 16377251434601:
		# 	print(len(data['shapes']))
		# 	print(len(single_img_cls))
		# 	print(len(single_img_anno))
		# 	print(img_idx)

		for anno, cls in zip(single_img_anno, single_img_cls):
			x_lt = anno[0]
			y_lt = anno[1]
			w = anno[2]
			h = anno[3]
			ang = anno[4]


			annotations.append(
				{
					"id": anno_idx,
					"category_id": cat_cvt[cls] + 1,
					"image_id": img_idx,
					"bbox": [
						x_lt-1,
						y_lt-1,
						w+2,
						h+2,
						ang
					],
					"conf": 1,
					"iscrowd": 0,
					"area": (w+2) * (h+2)
				})
			anno_idx += 1

		# if f == 16377251434601:
		# 	print(list(filter(lambda x: x['image_id'] == 8, annotations)))
		# 	print(len(list(filter(lambda x: x['image_id'] == 8, annotations))))

		img_idx += 1

	out = dict(
		images=images,
		annotations=annotations,
		categories=categories
	)

	with open('{}.json'.format(os.path.join(out_json_path, split_str[si])), "w") as f:
		json.dump(out, f, indent = 2)